# Exercise 1 - Pooling

In [ ]:
import numpy as np

## Objective

In this exercise, you will implement a simplified version of the max pooling layer.

## Details

You will have to implement two functions and a small script. The first function is a padding 
function. Using the input size and the pooling layer parameters (stride and filter size), 
this function finds the padding `wpad` and `hpad` (width and height padding) such that the 
input dimensions are padded.

In [ ]:
### From Udacity's `pooling.py`

In [ ]:
def get_paddings(array, pool_size, pool_stride):
    """ 
    get padding sizes 
    args:
    - array [array]: input np array NxwxHxC
    - pool_size [int]: window size
    - pool_stride [int]: stride
    returns:
    - paddings [list[list]]: paddings in np.pad format
    """
    # IMPLEMENT THIS FUNCTION
    wpad = None
    hpad = None
    return [[0, 0], [0, wpad], [0, hpad], [0, 0]]

The next function calculates the output dimensions after pooling given the padded array
dimensions and the pooling parameters (stride and filter size).

In [ ]:
### From Udacity's `pooling.py`

In [ ]:
def get_output_size(shape, pool_size, pool_stride):
    """ 
    given input shape, pooling window and stride, output shape 
    args:
    - shape [list]: input shape
    - pool_size [int]: window size
    - pool_stride [int]: stride
    returns
    - output_shape [list]: output array shape
    """
    # IMPLEMENT THIS FUNCTION
    new_w = None
    new_h = None
    return [shape[0], int(new_w), int(new_h), shape[3]]

Finally, the script calculates the pooling layer output.

In [ ]:
### From Udacity's `pooling.py`

In [ ]:
parser = argparse.ArgumentParser(description='Download and process tf files')
parser.add_argument('-f', '--pool_size', required=True, type=int, default=3,
                    help='pool filter size')
parser.add_argument('-s', '--stride', required=True, type=int, default=3,
                    help='stride size')
args = parser.parse_args()

input_array = np.random.rand(1, 224, 224, 16)
pool_size = args.pool_size
pool_stride = args.stride

# padd the input layer
paddings = get_paddings(input_array, pool_size, pool_stride)
padded = np.pad(input_array, paddings, mode='constant', constant_values=0)

# get output size
output_size = get_output_size(padded.shape, pool_size, pool_stride)
output = np.zeros(output_size)

# IMPLEMENT THE POOLING CALCULATION 
check_output(output)

You can run `python pooling.py` to check your implementation - note that the checking of the output will require input of a 3x3 filter and stride of 3.

In [ ]:
### From Udacity's `utils.py`

In [ ]:
def check_output(output):
    """
    checking the shape of the output
    """
    if output.shape == (1, 75, 75, 16):
        print('Success!')
    else:
        print('Failure')

## Tips

Pooling only affects the spatial dimensions and preserves the batch size (first axis of the padded array) 
and the number of channels (last axis).